In [0]:
!pip install scikit-fuzzy
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import random
import skfuzzy as fuzz
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler

     |████████████████████████████████| 1.0MB 2.8MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-cp36-none-any.whl size=894070 sha256=a2bafe58edfd6d2155e712e672cc782fceb66332454fa912e4b4a60d5ea797fa
  Stored in directory: /root/.cache/pip/wheels/b9/4e/77/da79b16f64ef1738d95486e2731eea09d73e90a72465096600
Successfully built scikit-fuzzy


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def potential(X,ra):
  
  alpha=4/ra
  P=[]
  for i in range(X.shape[0]):
    s=0
    for k in range(X.shape[0]):
      if i==k:
        continue
      else:
        s+=np.exp(-alpha*np.power(np.sum(X[i]-X[k]),2)/ra)
    P.append(s)
  
  return P

In [0]:
def rules(X,P,ra):
  M=max(P)
  cluster_center=[M]
  clust_ind=[P.index(M)]
  rb=1.5*ra
  beta=4/(rb**2)
  

  while cluster_center[-1]>(0.15*cluster_center[0]):

    
    # print(cluster_center)
    # print(clust_ind)
    for i in range(X.shape[0]):
      if i in clust_ind:
        continue
      else:
        # print(i)
        sub=np.exp(-beta*np.power(np.sum(X[i]-X[clust_ind[-1]]),2))
        P[i]-=M*sub
    # print(P)
    M=max(P)
    ind=P.index(M)
    dmin=-999
    P1=P.copy()
    clust_sort=sorted(clust_ind)
    for i in range(len(clust_sort)):
        if i==0:
          P1=P1[0:clust_sort[i]]+P1[clust_sort[i]+1:]
        else:
          P1=P1[0:clust_sort[i]-i]+P1[clust_sort[i]-i+1:]
  
        # print(P1[45:61])
    
    while (dmin/ra)+(M/cluster_center[0])<1:
      
      M=max(P1)
      ind=P.index(M)
      dmin=99999
      for j in range(len(clust_ind)):
        dist=abs(np.sum(cluster_center[j]-X[ind]))
        if dist<dmin:
          dmin=dist
      if (dmin/ra)+(M/cluster_center[0])>=1:
        cluster_center.append(M)
        clust_ind.append(ind)
        break
      else:
        
        P[ind]=0
        P1[P1.index(M)]=0
        
  return cluster_center,clust_ind,rb

In [0]:
def fmembership(cluster_center,X,clust_ind,rb):
  membership=[[0 for i in range(len(cluster_center))]for j in range(X.shape[0])]
  sigma=rb/(8**0.5)
  for i in range(X.shape[0]):
    for k in range(len(clust_ind)):
      
      membership[i][k]=np.exp(-(np.power(np.sum(X[i]-X[clust_ind[k]]),2))/(2*sigma*sigma))
  return membership,sigma

In [0]:
def genclass(cluster_center,clust_ind,y):
    classes={0.0:None,1.0:None}
    
    for i in range(len(clust_ind)):
        
        if classes[y[clust_ind[i]]]==None:
          classes[y[clust_ind[i]]]=[cluster_center[i]]
        else:
          classes[y[clust_ind[i]]].append(cluster_center[i])
    return classes

In [0]:
def pred(membership,X,classes,cluster_center):
  predict=[]
  for i in range(X.shape[0]):
    c=max(membership[i])
    pclass=membership[i].index(c) 
    if classes[0.0]!=None and cluster_center[pclass] in classes[0.0]:
      predict.append(0.0)
    else:
      predict.append(1.0)
  return predict

In [0]:
def accuracy(predict,y):
  c=0
  fp=0
  fn=0
  tn=0
  tp=0
  for i in range(len(predict)):
    if predict[i]==y[i]:
      c+=1
      if y[i]==0.0:
        tn+=1
      else:
        tp+=1
    elif predict[i]==0.0 and y[i]==1.0:
      fn+=1
    else:
      fp+=1
  return (c/len(predict)),tp,tn,fp,fn

In [0]:
import random
def init_pos(d):
  pos=[]
  for i in range(d):
    pos.append(random.random())
  return pos

In [0]:
def preprocess(theta,cols):
  exact_cols=[]
  exact_ch=[]
  for j in range(len(theta)):
      if theta[j]==1:
        exact_ch.append(str(cols[j]*5-4))
        exact_ch.append(str(cols[j]*5-3))
        exact_ch.append(str(cols[j]*5))
        exact_cols.append(j+1)
  return exact_cols,exact_ch

In [0]:
import statistics
def mean(X):
  me=[]
  for i in range(X.shape[1]):
    me.append(statistics.mean(X.iloc[:,i]))
  return me

In [0]:
import collections
def fitness(w,acc,theta,exact_cols,intense):
  zero=collections.Counter(theta)[0]
  for i in range(len(exact_cols)):
    intense[exact_cols[i]-1]=(acc+w[exact_cols[i]-1]*zero)
  return intense

In [0]:
import random
import xgboost as xgb
from sklearn import svm
from sklearn.metrics import accuracy_score
import math
import collections
def firefly(N,gamma,alpha,X,y,col_names):
  pos=init_pos(64)
  theta=[1]*64
  w=[0.5]*64
  exact_cols,exact_ch=preprocess(theta,col_names)
  
  intense=[1]*64
  Xuse=X[exact_ch].copy()
  Xuse=np.array(Xuse)
  clf=svm.SVC()
  
  clf.fit(Xuse,y)
  
  y_pred=clf.predict(Xuse)
  acc=accuracy_score(y, y_pred)
  print(acc)
  me=mean(X)
  
  for k in range(N):
    fire_d=len(pos)
    intense=fitness(w,acc,theta,exact_cols,intense)
    for i in range(len(exact_cols)):
      for j in range(len(exact_cols)):
        if i==j:
          continue
        elif theta[i]==0 or theta[j]==0:
          continue
        else:
          if intense[j]>=intense[i]:
            ri1=me[((exact_cols[i]-1)*5-5)]-me[((exact_cols[j]-1)*5-5)]
            ri2=me[((exact_cols[i]-1)*5-4)]-me[((exact_cols[j]-1)*5-4)]
            
            ri5=me[((exact_cols[i]-1)*5-1)]-me[((exact_cols[j]-1)*5-1)]
            r=math.sqrt(math.pow(ri1,2)+math.pow(ri2,2)+math.pow(ri5,2))
            pos[exact_cols[i]-1]=pos[exact_cols[i]-1]+0.2*math.exp(-gamma*math.pow(r,2))*(pos[exact_cols[j]-1]-pos[exact_cols[i]-1])+alpha*(random.uniform(0,1)-0.5)

    for i in range(fire_d):
      if pos[i]>0.55:
        w[i]+=0.01
      else:
        theta[i]=0
    # print(theta)
    exact_cols,exact_ch=preprocess(theta,col_names)
    
    Xuse=X[exact_ch].copy()
    Xuse=np.array(Xuse)
    clf.fit(Xuse,y)
    y_pred=clf.predict(Xuse)
    acc=accuracy_score(y, y_pred)
    print(acc)
  return exact_ch,exact_cols

In [0]:
def data_init():
  data=pd.read_csv('drive/My Drive/train.csv')
  data.drop(28,inplace=True)
  data.dropna()
  print(data.shape)
  X=data.iloc[:,0:320]
  y=data.iloc[:,335]
  # print(y)
  print(X.shape)
  return X,y

In [0]:
def data_norm(X):
  col_names=list(X.columns)
  temp=pd.DataFrame()
  for i in range(320):
      minimum=min(X[col_names[i]])
      maximum=max(X[col_names[i]])
      temp[col_names[i]]=((X[col_names[i]]-minimum)/(maximum-minimum))
  return temp,col_names

In [0]:
def fire_call(X,y):
  cols=list(range(1,65))
  exact_ch,exact_cols=firefly(100,1,0.5,X,y,cols)
  return exact_ch,exact_cols


In [0]:
def print_exact_ch(exact_ch,exact_cols):
  print(len(exact_ch))
  print(exact_cols)

In [0]:
def fire_init(X,exact_ch):
  Xuse=X[exact_ch].copy()
  print(Xuse.shape)
  return Xuse

(118, 39)


In [0]:
def run_sub(exact_ch,Xuse,y):
  r_acc={}
  r_cf={}
    
  for r in  range(1,10,1):
      print(r)
      ra=r/10
      acc={}
      cf={}
      for j in range(3,len(exact_ch)+1,3):
        X=Xuse.iloc[:,0:j+1]
        y=np.array(y)
        X=np.array(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
        
        # for ep in range(40):
          
        
        P=potential(X_train,ra)
        cluster_center,clust_ind,rb=rules(X_train,P,ra)
        membership,sigma=fmembership(cluster_center,X_train,clust_ind,rb)
        classes=genclass(cluster_center,clust_ind,y_train)
        predict=pred(membership,X_test,classes,cluster_center)
          # X_train=grad_desc(X_train,cluster_center,membership,classes,clust_ind,sigma)
        c,tp,tn,fp,fn=accuracy(predict,y_test)
          # if ep==39:
        acc[j]=c
        cf[j]={'tp':tp,'tn':tn,'fp':fp,'fn':fn}
      r_acc[r]=acc
      r_cf[r]=cf
  return r_acc,r_cf

In [0]:
def max_acc(r_acc):
  maxi=[]
  for i in range(1,len(r_acc.keys())):
    key_max = max(r_acc[i].keys(), key=(lambda k: r_acc[i][k]))
    maxi.append(r_acc[i][key_max])
  print(max(maxi))
  return max(maxi)

In [0]:
def wrapper():
  maxi=0
  max_r_acc=[]
  max_r_cf=[]
  while maxi<0.90:
    X,y=data_init()
    X,col=data_norm(X)
    exact_ch,exact_cols=fire_call(X,y)
    print_exact_ch(exact_ch,exact_cols)
    Xuse=fire_init(X,exact_ch)
    r_acc,r_cf=run_sub(exact_ch,Xuse,y)
    maxi=max_acc(r_acc)
    max_r_acc,max_r_cf=r_acc,r_cf
    print("Current r_acc:"+str(maxi))
  return max_r_acc,max_r_cf

In [0]:
max_r_acc,max_r_cf=wrapper()

(118, 336)
(118, 320)
0.788135593220339
0.7966101694915254
0.7966101694915254
0.7796610169491526
0.7796610169491526
0.7711864406779662
0.7711864406779662
0.7711864406779662
0.7711864406779662
0.7711864406779662
0.7711864406779662
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.7796610169491526
0.77966101

In [0]:
print(max_r_acc)
print(max_r_cf)

{1: {3: 0.5, 6: 0.5833333333333334, 9: 0.5833333333333334, 12: 0.5833333333333334, 15: 0.4583333333333333, 18: 0.7083333333333334, 21: 0.5416666666666666, 24: 0.5, 27: 0.375}, 2: {3: 0.375, 6: 0.5416666666666666, 9: 0.4583333333333333, 12: 0.9166666666666666, 15: 0.625, 18: 0.5416666666666666, 21: 0.5, 24: 0.4166666666666667, 27: 0.4166666666666667}, 3: {3: 0.5833333333333334, 6: 0.375, 9: 0.5, 12: 0.5833333333333334, 15: 0.6666666666666666, 18: 0.5, 21: 0.4166666666666667, 24: 0.4166666666666667, 27: 0.4583333333333333}, 4: {3: 0.6666666666666666, 6: 0.8333333333333334, 9: 0.2916666666666667, 12: 0.5833333333333334, 15: 0.625, 18: 0.5, 21: 0.4166666666666667, 24: 0.5416666666666666, 27: 0.375}, 5: {3: 0.5, 6: 0.6666666666666666, 9: 0.5416666666666666, 12: 0.6666666666666666, 15: 0.25, 18: 0.375, 21: 0.25, 24: 0.2916666666666667, 27: 0.4583333333333333}, 6: {3: 0.5, 6: 0.5833333333333334, 9: 0.5416666666666666, 12: 0.4583333333333333, 15: 0.7083333333333334, 18: 0.5416666666666666, 21: